# <p style="color:dodgerblue">Image Generation in Bedrock</p>
<hr style="border:1px dotted; color:floralwhite">

# Requirements for this lab
*See <span style="color:gold">Appendix</span> at the bottom of this lab to install requirements for this lab*

<hr style="border:1px dotted">
<hr style="border:1px dotted;color:cadetblue">

# <p style="color:cadetblue">Import libraries and set clients</p>
*If the following does not show the latest (installed or upgraded) boto3 (at least v1.29.6), restart the kernel*

In [ ]:
import boto3, json
boto3.__version__

In [ ]:
from io import BytesIO
from PIL import Image
import base64
import random

# region_name is important especially if you're aws configure is set to ap-southeast-2 where there is no Bedrock yet
#define common vars
myRegion='us-east-1'

bedrockRun = boto3.client(service_name='bedrock-runtime', region_name=myRegion)
bedrockChk = boto3.client(service_name='bedrock', region_name=myRegion)

<hr style="border:1px dotted;color:cadetblue">
<hr style="border:1px dotted;color:crimson">

# <p style="color:crimson">Create some methods to call</p>
1. Method for generating an image
2. <b>Don't forget if you modify this method, make sure you execute the cell</b>

In [ ]:
# create an image from text
def createImageFromText(m, prompt, np, s, c, h, w, n, st):
    # prep some vars
    body=''
    accept = "application/json"
    contentType = "application/json"

    # which model are we calling
    match m:
        case "amazon.titan-image-generator-v1":
            # Amazon - Titan Image Generator G1
            # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html
            body = json.dumps(
                {
                    "taskType": "TEXT_IMAGE",
                    "textToImageParams": {
                        "text": prompt,
                        "negativeText": np,
                    },
                    "imageGenerationConfig": {
                        "numberOfImages": n,
                        "quality": "premium",
                        "height": h,
                        "width": w,
                        "cfgScale": c,
                        "seed": s,
                    },
                }
            )
        case "stability.stable-diffusion-xl-v1":
            # Stability.ai - Diffusion models
            # https://stability.ai/
            body = json.dumps(
                {
                    "text_prompts": [
                        {
                            "text": prompt,
                            "weight": 1,
                        }
                    ],
                    "height": h,
                    "width": w,
                    "cfg_scale": c,
                    "seed": s,
                    "style_preset" : st,
                }
            )

    # invoke the model
    response = bedrockRun.invoke_model(
        body=body, modelId=m, accept=accept, contentType=contentType
    )
    responseBody = json.loads(response.get("body").read())

    match m:
        case "amazon.titan-image-generator-v1":
            # required to preview and save as png
            images1 = [
                Image.open(BytesIO(base64.b64decode(base64_image)))
                for base64_image in responseBody.get("images")
            ]

            # required if you want to pass into further invokes, in painting for example
            imagesBinary1 = [
                base64_image
                for base64_image in responseBody.get("images")
            ]

            # show the images - on a mac this will open the Preview app
            for img in images1:
                img.show()
        case "stability.stable-diffusion-xl-v1":
            # required to preview and save as png
            base64_image = responseBody.get("artifacts")[0].get("base64")

            # required to pass into further invokes
            base64_bytes = base64_image.encode('ascii')
            image_bytes = base64.b64decode(base64_bytes)

            # show the image - on a mac this will open the Preview app
            image = Image.open(BytesIO(image_bytes))
            image.show()

<hr style="border:1px dotted;color:crimson">
<hr style="border:1px dotted;color:lightblue">

# <p style="color:lightblue">Invoke a model with a text prompt</p>
1. Specify properties
2. Call the method

In [ ]:
# set up the prompt and params - note each model will have different ranges for its params
prompt='A vibrant and busy street in Bangkok is full of happy families walking through street vendors selling food and gifts. It is evening, and the streets are lit with soft lighting creating a cozy atmosphere. Photographic, ultra realistic, atmospheric, hyper detailed.'
negativePrompt='low resolution, low details, multiple views'
# NOTE if you receive an error, check the value of the properties you are providing as it is likely outside the range of the model
h=1024 # height
w=1024 # width
c=4 # cfgScale: determines how much the final image portrays the prompt, use a lower value for more random images
n=3 # numResults: how many images do you want, not all models support multiple images
st='photographic' # style preset that guides the image model towards a particular style,not all models support

# which model do you want to use based on the match statement below
u=2

match u:
    case 1:
        # Amazon - Titan Image Generator G1
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/titan-image-models.html
        m='amazon.titan-image-generator-v1'
        s = random.randint(0, 214783647) # random see of the image
    case 2:
        # Stability.ai Diffusion 1.0 text to image
        # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-diffusion-1-0-text-image.html
        m='stability.stable-diffusion-xl-v1'
        s = random.randint(0, 4294967295) # random see of the image

print("Seed is (in case you like the image!): {}".format(s))

createImageFromText(m, prompt, negativePrompt, s, c, h, w, n, st)


<hr style="border:1px dotted;color:lightblue">
<hr style="border:1px dotted;color:gold">

# <p style="color:gold">Appendix - Install Requirements (macOS)</p>
# Requirements for this lab

#### <p style="color:deeppink">- If you are running VSCode on a laptop, follow all of below.<br>- If you are running Jupyter inside an AWS Account, you just need to install pillow. See 2.1 below and skip everything else.</p>

*Windows requirements will be similar, apart from Homebrew.*  
* python must be installed on your client, and be at least v3.8
  * 3.8 supports the latest release of boto3 which supports Bedrock  
* boto3 must be installed on your client, and be at least v1.33.9  
  * *Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.*
  * https://boto3.amazonaws.com/v1/documentation/api/latest/index.html
* 
* Bedrock model access requested
  * Models used in this lab with access already requested are:
    * AI21 Labs - Jurassic-2 Ultra and Jurassic-2 Mid
    * Meta Llama 2 Chat 13B
### <p style="color:gold">1. Homebrew</p> 
If you haven't installed Homebrew, you can install it by running the following command here or in the terminal:

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1. Python</p> 
Once Homebrew is installed, you can install Python using the following command  
*check what you have before installing/upgrading*  
*you will need to quit and restart vsCode to use python once installed (or updated)*

In [ ]:
%%bash
python3 --version
which python3

In [ ]:
%%bash
brew install python

### <p style="color:gold">2. boto3 and other Python requirements</p> 
*check what you have before installing/upgrading*  

In [ ]:
%%bash
python3 -m pip show boto3

In [ ]:
pip install -U boto3

### <p style="color:gold">2.1 Image requirements</p> 
*<p style="color:deeppink">If you are running a Jupyter in AWS Account, you ONLY need this</p>*

In [ ]:
pip install -U pillow

### <p style="color:gold">3. aws configure</p> 
*Configure aws configure with credentials, and a user that has all of the Bedrock IAM policies required*  
https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html

In [ ]:
%%bash
aws sts get-caller-identity

### <p style="color:gold">4. Request Bedrock model access</p> 
*You must request access to the models required, you may need to provide use case details before you are able to request*  
*Make sure you request in the region you intend to use the models in, this lab is us-east-1*  
https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/modelaccess  

Models required in this lab:

* See code above for use of models and what access to request

#### Pricing
*Price is calculated based on properties including quality of the image.*  
https://aws.amazon.com/bedrock/pricing/  
https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-prepare.html

### <p style="color:gold">5. Image editor that can create masks</p> 

https://www.gimp.org/downloads/

<hr style="border:1px dotted;color:gold">